In [ ]:
import pandas as pd

In [ ]:
import google.generativeai as genai

genai.configure(api_key="api_key")

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
!pip install cerebras_cloud_sdk

In [ ]:
from cerebras.cloud.sdk import Cerebras

client = Cerebras(
  api_key="api-cerebras",
)

#### CARER dataset

In [ ]:
carer = pd.read_csv("./data.csv")
emotion_map = {
    0: 'Sadness',
    1: 'Joy',
    2: 'Love',
    3: 'Anger',
    4: 'Fear',
    5: 'Surprise'
}
carer["label"] = carer["label"].map(emotion_map)
carer = carer[carer["label"] != "Love"]
carer.reset_index(drop=True, inplace=True)
carer.head()

,text,label
0,i feel awful about it too because it s my job ...,Sadness
1,im alone i feel awful,Sadness
2,ive probably mentioned this before but i reall...,Joy
3,i was feeling a little low few days back,Sadness
4,i am one of those people who feels like going ...,Joy


#### REN-20k dataset

In [ ]:
ren_20k_filtered = pd.read_csv("ren_20k_filtered.csv")

In [ ]:
PROMPT_GENERATOR='''Please decide whether the emotion of the input sentence is sadness, joy, anger, fear or surprise. You must choose one of those and not any other! Take the input as it is don't try to modify it! I will give you a few shot demonstration. Start your response directly with the identified emotion and keep the answer in one paragraph'''
FEW_SHOT_DEM_GENERATOR='''Please decide whether the emotion of the input sentence is sadness, joy, anger, fear or surprise.
FEW-SHOT DEMONSTRATION:
INPUT: "I can't believe this game I am never playing it again" Give an explanation of your decision-making process. Make the response in one paragraph.
GENERATOR: Anger. The sentence expresses frustration and dissatisfaction, particularly with the use of "I can't believe," which conveys disbelief, and "never playing it again," indicating a strong decision based on negative feelings. This suggests a reaction to something upsetting or disappointing, likely related to the game experience. There is no indication of happiness, affection, fear, or surprise, making anger the most fitting emotion as it conveys an emotional response to a frustrating situation.

'''
PROMPT_DISCRIMINATOR='''I will give you an example of a generator and discriminator conversation to decide an emotion from a text'''
FEW_SHOT_DEM_DISCRIMINATOR='''Do you agree with the generator's decision that the emotion of the input is joy?

INPUT: "If you sometimes like to go to the movies to have fun, Wasabi is a good place to start"

GENERATOR: Joy. The sentence conveys a cheerful and positive mood by suggesting an enjoyable activity—going to the movies for fun. The recommendation of "Wasabi" as a good starting point reinforces a sense of enthusiasm and lightheartedness. There is no mention of negative experiences or emotions such as sadness, anger, or fear, nor does it express love or romantic affection. Additionally, there is no element of shock or unexpectedness that would indicate surprise. The overall tone is upbeat and encouraging, making joy the most fitting emotion.

DISCRIMINATOR: Yes, I agree with the generator's decision that the emotion of the input is joy. The sentence conveys a cheerful and positive mood by suggesting an enjoyable activity—going to the movies for fun. The recommendation of "Wasabi" as a good starting point reinforces a sense of enthusiasm and lightheartedness. There is no mention of negative experiences or emotions such as sadness, anger, or fear, nor does it express love or romantic affection. Additionally, there is no element of shock or unexpectedness that would indicate surprise. The overall tone is upbeat and encouraging, making joy the most fitting emotion.Joy.

'''
PROMPT_DISCRIMINATOR_FINAL='''The example has ended here. I need you to play the role of the discriminator for the following input given. The emotions the generator chose from and you have available as well are: sadness, joy, anger, fear or surprise. Keep the answer structure as presented in the example. Add a sentence at the end of your response that contains the emotion you identified. Now i will give you the question: Do you agree with the generator's decision that the emotion of the input is: '''
INPUT_DISCRIMINATOR="INPUT: "+carer['text'][0]

CONV_PROMPT='''This is the conversation you have been involved in. Do you agree with the last thing that was said regarding the identified emotion? Start your answer with No or Yes add a comma, give your reasoning for agreeing or disagreeing add a period and then state the emotion you have decided on using a single word. Look at last given answer to understand the structure "'''


In [ ]:
import time
right=0
wrong=0

output = pd.DataFrame(columns=["Emotion"])

for i in range(10000):
  CONVERSATION=""
  INPUT_GENERATOR="INPUT: "+carer['text'][i]
  to_gen=PROMPT_GENERATOR+FEW_SHOT_DEM_GENERATOR+INPUT_GENERATOR
  CONVERSATION=PROMPT_GENERATOR+INPUT_GENERATOR
  chat_completion = client.chat.completions.create(
  messages=[
  {"role": "user", "content": to_gen}
  ],
  model="llama3.1-8b",
  )
  chat_response_content = chat_completion.choices[0].message.content
  CONVERSATION+=chat_response_content
  EMOTION=chat_response_content
  to_gen_discriminator=PROMPT_DISCRIMINATOR+FEW_SHOT_DEM_DISCRIMINATOR+PROMPT_DISCRIMINATOR_FINAL+EMOTION+INPUT_GENERATOR+chat_response_content
  DISCRIMINATOR_RESPONSE=model.generate_content(to_gen_discriminator).text
  CONVERSATION+=DISCRIMINATOR_RESPONSE
  if "Yes" in DISCRIMINATOR_RESPONSE.split(',')[0]:
    DECIDED_EMOTION=chat_response_content.split('.')[0]
  else:
    for j in range(0,3):
      chat_completion1 = client.chat.completions.create(
      messages=[
      {"role": "user", "content": CONVERSATION+CONV_PROMPT}
      ],
      model="llama3.1-8b",
      )
      chat_response_content1 = chat_completion1.choices[0].message.content
      CONVERSATION+=chat_response_content1
      response=chat_response_content1.split(',')[0]
      if response=='Yes':
        DECIDED_EMOTION=DISCRIMINATOR_RESPONSE.split('.')[-2]
        print("Dezacord"+DECIDED_EMOTION)
        break
      CONVERSATION+=chat_response_content1
      next_ans=CONVERSATION+CONV_PROMPT
      rez=model.generate_content(next_ans).text
      if rez.split(",")[0]=='Yes':
        DECIDED_EMOTION=rez.split('.')[-2]
        break
      CONVERSATION+=rez
    DECIDED_EMOTION=DISCRIMINATOR_RESPONSE.split('.')[-2]
  DECIDED_EMOTION = DECIDED_EMOTION.replace(" ", "")
  output = pd.concat([output, pd.DataFrame([{"Emotion": DECIDED_EMOTION}])], ignore_index=True)
  if(DECIDED_EMOTION==carer['label'][i]):
    right+=1
  else:
    wrong+=1
  time.sleep(5.5)

print(right,wrong)

Dezacord Sadness
Dezacord
Self-Reflection
Dezacord Joy
Dezacord Joy
Dezacord

The emotion identified is more accurately described as neither joy, sadness, anger, fear, nor surprise
Dezacord Sadness
Dezacord  Sadness
Dezacord

Sadness
Dezacord Sadness
Dezacord Joy
Dezacord  The emotion is more likely to be indifference
Dezacord
Anger
Dezacord Sadness
Dezacord  Sadness is not the most fitting emotion
Dezacord
Relief
Dezacord Fear
Dezacord Surprise
Dezacord  Sadness
Dezacord Joy
Dezacord Joy
Dezacord  Hope
Dezacord Disappointment
Dezacord  Joy
Dezacord Sadness
Dezacord  Therefore, a more fitting emotion would be joy
Dezacord
Fear
Dezacord  Anger
Dezacord Sadness
Dezacord Joy
Dezacord  Relief
Dezacord Anger
Dezacord  The sentence expresses **shyness**
Dezacord  The emotion identified is guilt
113 110


In [ ]:
output.to_csv('carer_output.csv', index=False)

In [ ]:
PROMPT_GENERATOR = '''
You must decide the primary emotion expressed in the given input sentence. The available emotions are: sadness, joy, anger, fear, and surprise. Choose only one, and do not include any other! Do not alter the input sentence. Begin your response directly with the identified emotion and justify your decision in one paragraph.
'''
FEW_SHOT_DEM_GENERATOR = '''
Here is an example of how to decide the emotion. Keep the structure of the response like the one in this example:

INPUT: "I can't believe this game I am never playing it again"
RESPONSE: Anger. The sentence shows frustration and dissatisfaction, evident from "I can't believe," expressing disbelief, and "never playing it again," indicating a strong emotional reaction. There is no indication of joy, sadness, fear, or surprise, making anger the most appropriate emotion here.

Now, decide for the input given below:
'''
PROMPT_DISCRIMINATOR = '''
I will show you a conversation between a generator and a discriminator where the goal is to evaluate the generator's emotion decision. Your task is to assess whether the generator's choice is accurate based on the input and explanation provided.
'''
FEW_SHOT_DEM_DISCRIMINATOR = '''
Example:

INPUT: "If you sometimes like to go to the movies to have fun, Wasabi is a good place to start"

GENERATOR: Joy. The sentence suggests a lighthearted and cheerful mood, emphasizing fun and enjoyment with a recommendation for movies. No indications of sadness, anger, fear, or surprise are present, supporting joy as the fitting choice.

DISCRIMINATOR: Yes, I agree with the generator's decision that the emotion of the input is joy. The explanation highlights positive and cheerful elements, and there is no evidence of conflicting emotions. Therefore, joy is the most suitable choice. Joy.
'''

PROMPT_DISCRIMINATOR_FINAL = '''
The example ends here. Now, you must evaluate the generator's decision for the following input. The available emotions are: sadness, joy, anger, fear, and surprise. Keep your response in the structure of the example above. At the end, clearly state the final emotion you identify.
Do you agree with the generator's decision that the emotion of the input is:
'''

INPUT_DISCRIMINATOR = "INPUT: " + carer['text'][0]

CONV_PROMPT = '''
This is the ongoing conversation. Do you agree with the last response regarding the identified emotion? Start with Yes or No, add a comma, provide your reasoning in one concise sentence, and end with the identified emotion as a single word.
Example structure: "Yes, the explanation aligns with the input context and supports the identified emotion. Joy."
'''

In [ ]:
def get_data(start, end):
  return carer['text'][start:end]

In [ ]:
# Define valid emotions
VALID_EMOTIONS = {"sadness", "joy", "anger", "fear", "surprise"}

# Negotiation function
def negotiate_emotion(input_text, generator_model, discriminator_model, max_retries=3):
    """
    Perform emotion detection negotiation using an LLM generator and discriminator.
    :param input_text: The text input for emotion detection.
    :param generator_model: The model used for generating emotion predictions.
    :param discriminator_model: The model used for evaluating the generator's predictions.
    :param max_retries: Maximum negotiation rounds if generator and discriminator disagree.
    :return: Final decided emotion or None if no agreement is reached.
    """
    conversation = f"INPUT: {input_text}\n"

    # Step 1: Generator identifies emotion
    generator_prompt = (
        "Please decide whether the emotion of the input sentence is sadness, joy, anger, fear, or surprise. "
        "Respond as 'Emotion: [emotion]'.\n"
        f"{conversation}"
    )
    chat_completion = generator_model.chat.completions.create(
        messages=[{"role": "user", "content": generator_prompt}],
        model="llama3.1-8b",
    )
    generator_response = chat_completion.choices[0].message.content
    print(f"Generator Response: {generator_response}")
    emotion = generator_response.split(":")[-1].strip().lower()

    # Validate generator response
    if emotion not in VALID_EMOTIONS:
        print(f"Invalid emotion from generator: {emotion}")
        return None

    # Step 2: Discriminator evaluates the generator's emotion
    discriminator_prompt = (
        f"Do you agree with the generator's decision that the emotion is {emotion}?\n"
        f"INPUT: {input_text}\n"
        "If yes, explain why. If no, provide an alternative emotion."
    )
    discriminator_response = discriminator_model.generate_content(discriminator_prompt).text
    print(f"Discriminator Response: {discriminator_response}")

    # Extract decision and rationale
    agrees = discriminator_response.split(",")[0].strip().lower() == "yes"
    if agrees:
        return emotion

    # Step 3: Negotiation loop if disagreement
    for retry in range(max_retries):
        # Extract alternative emotion proposed by the discriminator
        alternative_emotion = discriminator_response.split(".")[-1].strip().lower()
        if alternative_emotion in VALID_EMOTIONS:
            emotion = alternative_emotion

        # Retry with the updated emotion
        retry_prompt = (
            f"The discriminator disagreed. Generator, do you want to adjust your decision?\n"
            f"Current proposed emotion: {emotion}. Explain your reasoning."
        )
        chat_completion_retry = generator_model.chat.completions.create(
            messages=[{"role": "user", "content": retry_prompt}],
            model="llama3.1-8b",
        )
        retry_response = chat_completion_retry.choices[0].message.content
        print(f"Retry {retry + 1} Generator Response: {retry_response}")

        # Validate retry response
        emotion = retry_response.split(":")[-1].strip().lower()
        if emotion not in VALID_EMOTIONS:
            print(f"Invalid emotion during retry: {emotion}")
            return None

        # Final check with discriminator
        discriminator_response = discriminator_model.generate_content(discriminator_prompt).text
        print(f"Retry {retry + 1} Discriminator Response: {discriminator_response}")
        agrees = discriminator_response.split(",")[0].strip().lower() == "yes"
        if agrees:
            return emotion

    # If no agreement is reached
    print("No agreement reached after maximum retries.")
    return None

# Test the negotiation process
test_inputs = [
    "I can't believe this is happening to me.",
    "What a wonderful surprise!",
    "I feel so sad about this situation.",
    "This is the scariest thing I've ever seen!",
    "I am so furious right now!"
]

# test_input = get_data(0,2500)

results = {"correct": 0, "incorrect": 0}
output = pd.DataFrame(columns=["Emotion"])
expected_emotions = ["sadness", "surprise", "sadness", "fear", "anger"]  # Mock expected labels

for i, text in enumerate(test_inputs):
    print(f"\n--- Test {i + 1}: {text} ---")
    detected_emotion = negotiate_emotion(
        input_text=text,
        generator_model=client,
        discriminator_model=model,
        max_retries=3
    )
    print(f"Final Detected Emotion: {detected_emotion}\n")
    # Evaluate results
    output = pd.concat([output, pd.DataFrame([{"Emotion": DECIDED_EMOTION}])], ignore_index=True)
    if detected_emotion == expected_emotions[i]:
        results["correct"] += 1
    else:
        results["incorrect"] += 1

print(f"Results: {results}")



--- Test 1: I can't believe this is happening to me. ---
Generator Response: Emotion: Sadness or Fear (likely a mix of both), but leaning more towards sadness, as the phrase 'can't believe this is happening to me' suggests a sense of loss, hopelessness, and resignation, which are characteristic of sadness.
Invalid emotion from generator: sadness or fear (likely a mix of both), but leaning more towards sadness, as the phrase 'can't believe this is happening to me' suggests a sense of loss, hopelessness, and resignation, which are characteristic of sadness.
Final Detected Emotion: None


--- Test 2: What a wonderful surprise! ---
Generator Response: Emotion: Surprise
Discriminator Response: Yes, I agree with the generator's decision that the emotion is surprise.  The phrase "What a wonderful surprise!" explicitly states a surprising event, and the adjective "wonderful" indicates a positive reaction to that surprise.  The exclamation mark further emphasizes the emotional intensity and su